# Wikipedia Network Analysis
## Full Project by Forrest Hangen

In [56]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk

from bs4 import BeautifulSoup as bs
import requests
import time
import random

import re

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import MinMaxScaler

# Step 1. Training Model to Identify People in Wikipedia Article Summary
(This will use the data and model I developed in Text Classification - Identifying People.ipynb. I did not label all the data I gathered (60000+ article summaries, but if anyone wants to continue labeling, be my guest!))

In [27]:
df_label = pd.read_csv('wiki_desc_dataset_no_duplicates.csv')

In [28]:
df_label['LABEL'].value_counts().loc['PERSON'] /sum(df_label['LABEL'].value_counts())

0.32715462610899876

In [29]:
df_labled = pd.read_csv('wiki_desc_dataset_no_duplicates.csv')
df_labled_only = (df_labled.dropna()
                            .drop(columns= 'Unnamed: 0'))
df_labled_only['LABEL'] = df_labled_only['LABEL'].apply(lambda x: 1 if x == 'PERSON' else 0)
df_labled_only.head()

,LABEL,Titles,Text
0,1,Charles H. Percy,"Charles Harting Percy (September 27, 1919 – Se..."
1,0,Head of state,A head of state (or chief of state) is the pub...
2,1,Henry Winkler,"Henry Franklin Winkler (born October 30, 1945)..."
3,0,La Violencia,La Violencia (Spanish pronunciation: [la βjoˈl...
4,0,London Buses route 176,London Buses route 176 is a Transport for Lond...


In [67]:
X_train, X_test, y_train, y_test = train_test_split(df_labled_only['Text'], 
                                                    df_labled_only['LABEL'], 
                                                    random_state=42, train_size= .95)
len(X_train), len(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(5996, 316)

In [72]:
count_final = CountVectorizer(lowercase=False, min_df = 20, ngram_range=(1,3)).fit(X_train)

X_train_matrix = count_final.transform(X_train)
X_test_matrix = count_final.transform(X_test)

log_final = LogisticRegression(solver = 'newton-cg').fit(X_train_matrix, y_train)

In [73]:
y_train_finalpred = log_final.predict(X_train_matrix)
y_test_finalpred = log_final.predict(X_test_matrix)

print('TRAIN')
print('Log Reg:\nAccuracy: {0}\nROC AUC: {1}'.format(accuracy_score(y_train, y_train_finalpred),
                                                     roc_auc_score(y_train, y_train_finalpred)))
print(confusion_matrix(y_train, y_train_finalpred))
print()
print('TEST')
print('Log Reg:\nAccuracy: {0}\nROC AUC: {1}'.format(accuracy_score(y_test, y_test_finalpred),
                                                     roc_auc_score(y_test, y_test_finalpred)))
print(confusion_matrix(y_test, y_test_finalpred))

TRAIN
Log Reg:
Accuracy: 0.9998332221480988
ROC AUC: 0.9998759612999255
[[4030    1]
 [   0 1965]]

TEST
Log Reg:
Accuracy: 0.9778481012658228
ROC AUC: 0.9837962962962963
[[209   7]
 [  0 100]]


This level of accuracy is perfect for what I'm trying to do here.

# Step 2. Gathering the Data!